In [1]:
# %matplotlib notebook
# %load_ext autoreload
# %autoreload 2

import os
import random

from operator import itemgetter

import cv2
import numpy as np
import torch
import torch.optim as optim
import matplotlib.pyplot as plt

from albumentations import (
    CLAHE,
    Blur,
    OneOf,
    Compose,
    RGBShift,
    GaussNoise,
    RandomGamma,
    RandomContrast,
    RandomBrightness,
)

from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import MultiStepLR
from albumentations.pytorch.transforms import ToTensorV2
from albumentations.augmentations.transforms import HueSaturationValue
from albumentations.augmentations.transforms import Normalize

from trainer import Trainer, hooks, configuration
from detector import Detector
from trainer.utils import patch_configs
from trainer.utils import setup_system
from detection_loss import DetectionLoss
from trainer.encoder import (
    DataEncoder,
    decode_boxes,
    encode_boxes,
    generate_anchors,
    generate_anchor_grid,
)
from trainer.metrics import APEstimator
from trainer.datasets import ListDataset
# from trainer.data_set_downloader import DataSetDownloader 
from trainer.matplotlib_visualizer import MatplotlibVisualizer

/home/rahul/.conda/envs/opcv/lib/python3.6/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
dataloader_config, trainer_config = patch_configs(epoch_num_to_set=10, batch_size_to_set=1)
dataset_config = configuration.DatasetConfig(
    root_dir="../../../Datasets/Road_Scene_Object_Detection",
    train_transforms=[
        RandomBrightness(p=0.5),
        RandomContrast(p=0.5),
        OneOf([
            RandomGamma(),
            HueSaturationValue(hue_shift_limit=20, sat_shift_limit=50, val_shift_limit=50),
            RGBShift()
        ],
            p=1),
        OneOf([Blur(always_apply=True), GaussNoise(always_apply=True)], p=1),
        CLAHE(),
        Normalize(),
        ToTensorV2()
    ]
)

In [4]:

dataset_train = ListDataset(
    root_dir=dataset_config.root_dir,
    data_dir = 'export',
    list_file='annots_converted_train.txt',

    classes=[
        "__background__",
        "biker",
        "car",
        "pedestrian",
        "trafficLight",
        "trafficLight-Green",
        "trafficLight-GreenLeft",
        "trafficLight-Red",
        "trafficLight-RedLeft",
        "trafficLight-Yellow",
        "trafficLight-YellowLeft",
        "truck"
    ],
    mode='train',
    transform=Compose(dataset_config.train_transforms),
    input_size=300
)

loader_train = DataLoader(
    dataset=dataset_train,
    batch_size=dataloader_config.batch_size,
    shuffle=True,
#     collate_fn=dataset_train.collate_fn,
    num_workers=dataloader_config.num_workers,
    pin_memory=True
)

In [5]:
iterator = iter(loader_train)
one_batch = next(iterator)

In [6]:
len(one_batch)

3

In [7]:
one_batch = next(iterator)

In [8]:
one_batch[1].shape

torch.Size([1, 17451, 4])

In [9]:
one_batch[2].shape

torch.Size([1, 17451])

In [10]:
one_batch[2]

tensor([[0, 0, 0,  ..., 0, 0, 0]])